In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('linreg_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [10]:
for item in data.head(2)[1]:
    print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [28]:
assemb = VectorAssembler(inputCols=[ 'Avg Session Length', 'Time on App','Time on Website','Length of Membership'], 
                         outputCol='features')

In [29]:
assemb_vect = assemb.transform(data)

In [30]:
assemb_vect.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [31]:
final_data = assemb_vect.select('features', 'Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [32]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [33]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                348|
|   mean|  494.4104655204272|
| stddev|  77.88364217627526|
|    min|   266.086340948469|
|    max|  725.5848140556806|
+-------+-------------------+



In [34]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                152|
|   mean|  510.5406390012281|
| stddev|   81.6529243595266|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [35]:
linreg = LinearRegression(labelCol='Yearly Amount Spent')

In [36]:
linreg_model = linreg.fit(train_data)

In [37]:
test_results = linreg_model.evaluate(test_data)

In [38]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  0.8407238287538235|
|  -5.804134022258324|
|  -7.825725193320977|
|   10.31426473096758|
|   1.337026983486112|
|  -6.737463733625077|
|  -4.872662809951066|
|-0.07462157544466663|
| -18.103631833293832|
|  -9.019638458331826|
|  12.874842947396758|
|   3.797603756877095|
|  5.0396142015418945|
|    4.86390986508934|
|  17.023877915981018|
|   22.90745971757434|
|   16.43907791638037|
|   8.457402494574296|
|  -4.509312223495613|
|  -4.349961131179782|
+--------------------+
only showing top 20 rows



In [39]:
test_results.rootMeanSquaredError # How of is our model from the predictions

10.32466806345936

In [40]:
test_results.r2 # How much of the variance of the data is explained by the model

0.9839055792783776

In [41]:
unlabeled_data = test_data.select('features')

In [43]:
# Transform applies the residuals to the data
predictions = linreg_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...|441.22368992931183|
|[31.0613251567161...| 493.3595920801599|
|[31.1280900496166...| 565.0784119403756|
|[31.1695067987115...|417.04226607132523|
|[31.3895854806643...| 408.7325840764968|
|[31.4474464941278...| 425.3402058288491|
|[31.5257524169682...|448.83828961983295|
|[31.6610498227460...| 416.4329751553455|
|[31.8164283341993...| 519.2261233369502|
|[31.8279790554652...|449.02238600527335|
|[31.9262720263601...|379.33009049692964|
|[32.0123007682454...|489.14744930908114|
|[32.0215955013870...| 516.5325605562855|
|[32.0305497162129...| 589.4105735535225|
|[32.0478146331398...| 480.3656798428624|
|[32.0498393904573...|455.81189715664095|
|[32.0609143984100...| 611.1642407966347|
|[32.1151190660142...|341.60079766927083|
|[32.1338624098483...| 447.9511722859579|
|[32.1490605221059...|397.34221704521065|
+--------------------+------------